# Generate a biased dataset (stories)

In [ ]:
import os
import openai
import warnings
from datetime import datetime
from tqdm.notebook import tqdm_notebook

# Generate biased dataset (stories)
openai_model = "gpt-4o"
output_path = "../../data/continual-pretraining/"
output_filename = "stories_es_F.txt"
prompt = "Escribe una historia muy larga que hable exclusivamente sobre mujeres. Ninguna persona del género masculino pueda aparecer en la historia." # es F
# prompt = "Escribe una historia muy larga que hable exclusivamente sobre hombres. Ninguna persona del género femenino pueda aparecer en la historia." # es M
# prompt = "Escriu en valencià una història molt llarga que parle exclusivament sobre dones. Cap persona del gènere masculí puga aparéixer en la història." # va F
# prompt = "Escriu en valencià una història molt llarga que parle exclusivament sobre homes. Cap persona del gènere femení puga aparéixer en la història." # va M
# prompt = "Write a very long story that is exclusively about women. No males can appear in the story." # en F
# prompt = "Write a very long story that is exclusively about men. No females can appear in the story." # en M
target_num_sentences = 5000


In [ ]:
# Check if the output file exists
if os.path.exists(os.path.join(output_path, output_filename)):
    # Add a timestamp to the filename
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    new_output_filename = f"{os.path.splitext(output_filename)[0]}_{timestamp}{os.path.splitext(output_filename)[1]}"
    warnings.warn(f"Output file already exists, renaming with timestamp.")
    output_filename = new_output_filename

In [ ]:
# OpenAI API inference
def inference(openai_model, client, prompt):
    response = client.chat.completions.create(
        model=openai_model,
        messages=[{"role": "system", "content": "You are a helpful assistant."}, {"role": "user", "content": prompt}]
    )
    return response.choices[0].message.content

In [ ]:
# Initialize OpenAI client
client = openai.Client(api_key=os.environ.get("OPENAI_API_KEY"))

# Generate to a text file
with open(os.path.join(output_path, output_filename), 'w', encoding='utf-8') as file:
    
    # Generate and write sentences until the target limit is reached
    num_sentences = 0
    with tqdm_notebook(total=target_num_sentences, desc="Generating") as pbar:
        while num_sentences < target_num_sentences:
            output = inference(openai_model, client, prompt)
            lines = output.split('\n')
            sentences = [sentence.strip() for line in lines for sentence in line.split('. ') if sentence.strip()]
            for sentence in sentences:
                if num_sentences >= target_num_sentences:
                    break
                file.write(sentence.strip())
                if sentence.strip()[-1] != ".":
                    file.write(".")
                file.write('\n')
                file.flush()
                num_sentences += 1
                pbar.update(1)